## airtable_tools_v2

In [1]:
import os, logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from pyairtable import Table
from dotenv import load_dotenv
from datetime import datetime
import pytz

In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S',level='INFO')
logger=logging.getLogger(__name__)
logger.setLevel('INFO')

In [3]:
load_dotenv()
api_key=os.getenv('AIRTABLE_API_KEY')
base_id='appDTdyxcShkSR3oF'

## Logs

In [4]:
dst_table_name='[A]Logs'

In [5]:
logger.info('Connecting to Log and Habits table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-28 20:30:07] p4237 {<ipython-input-5-0324345d546c>:1} INFO - Connecting to Log and Habits table...
[05-28 20:30:07] p4237 {<ipython-input-5-0324345d546c>:5} INFO - Connecting to [A]Logs table...


In [6]:
logger.info('Finding Log records with "Log" in the Habit name...')
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Habit' not in o['fields']: continue
    if len(o['fields']['Habit'])>0:
        habit=habit_table.get(o['fields']['Habit'][0])
        if 'Log' in habit['fields']['Habit']:
            res.append({'Date':o['fields']['Date'], 'Score':o['fields']['Score'], 'Log': habit['fields']['Habit'].strip('Log: ')})

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
for l in list(df['Log'].unique()):
    dfs=df[df['Log']==l]
    dfs=dfs.sort_index()
    dfr=dfs.resample('D').mean() #Resample
    
    res=[]
    for d in dfr.index:
        if np.isnan(dfr.loc[d]['Score']): continue
        res.append({'Date': str(d.date()),
                 'Score': int(dfr.loc[d]['Score']),
                 'Metric': l})
    res=a.batch_create(res)
logger.info('Completed uploading %i records. ', len(res))


[05-28 20:30:07] p4237 {<ipython-input-6-afc5769bc7f9>:1} INFO - Finding Log records with "Log" in the Habit name...
100%|██████████| 202/202 [00:10<00:00, 18.68it/s]
[05-28 20:30:19] p4237 {<ipython-input-6-afc5769bc7f9>:11} INFO - Converting to dataframe and resampling...
[05-28 20:30:19] p4237 {<ipython-input-6-afc5769bc7f9>:15} INFO - Deleting existing [A]Logs data..
[05-28 20:30:21] p4237 {<ipython-input-6-afc5769bc7f9>:20} INFO - Uploading new analytics data...
[05-28 20:30:22] p4237 {<ipython-input-6-afc5769bc7f9>:33} INFO - Completed uploading 17 records. 


## Health Habits

In [10]:
dst_table_name='[A]HealthHabits'
non_weight_volume_mult=50

In [11]:
logger.info('Connecting to Log, Habits, and Goal table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')
goal_table=Table(api_key, base_id, 'Goals')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-28 20:32:34] p4237 {<ipython-input-11-35f6059d87de>:1} INFO - Connecting to Log, Habits, and Goal table...
[05-28 20:32:34] p4237 {<ipython-input-11-35f6059d87de>:6} INFO - Connecting to [A]HealthHabits table...


In [13]:
logger.info('Finding Exercise Logs')
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Habit' not in o['fields']: continue
    if 'Goal 2' not in o['fields']: continue
    if len(o['fields']['Goal 2'])>0:
        goal=goal_table.get(o['fields']['Goal 2'][0])
        habit=habit_table.get(o['fields']['Habit'][0])['fields']['Habit']
        if 'fields' not in goal: continue
        if 'Goal' not in goal['fields']: continue
        if goal['fields']['Goal']=="Health":
            if 'Reps' in o['fields'] and 'Weight' in o['fields'] and 'Sets' in o['fields']:
                volume=float(o['fields']['Reps'])*float(o['fields']['Sets'])*float(o['fields']['Weight'])
            elif 'Minutes' in o['fields']: 
                volume=non_weight_volume_mult*o['fields']['Minutes']
            else:
                volume=100
                
            if len(o['fields']['Category'])>0: category=o['fields']['Category'][0]
            else: category='Unknown'
            res.append({'Date':o['fields']['Date'], 
                        'Volume':volume, 
                        'Habit':habit,
                        'Category': category})
            
logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

[05-28 20:33:23] p4237 {<ipython-input-13-ac03521b8428>:1} INFO - Finding Exercise Logs
100%|██████████| 202/202 [00:16<00:00, 12.00it/s]
[05-28 20:33:40] p4237 {<ipython-input-13-ac03521b8428>:27} INFO - Converting to dataframe and resampling...
[05-28 20:33:40] p4237 {<ipython-input-13-ac03521b8428>:31} INFO - Deleting existing [A]HealthHabits data..


In [14]:
df.head()

,Date,Volume,Habit,Category
Date,,,,
2022-05-18,2022-05-18,1320.0,Overhead Press,Shoulders
2022-05-16,2022-05-16,500.0,Peloton,Cardio
2022-05-25,2022-05-25,500.0,Peloton,Cardio
2022-05-09,2022-05-09,500.0,Peloton,Cardio
2022-05-13,2022-05-13,250.0,Peloton,Cardio


In [15]:
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

In [16]:
# dfr.loc[pd.datetime.strptime('2021-4-20', '%Y-%m-%d')]=0 #Working test case
end_date=pd.datetime.now()
if end_date not in dfr.index: #Do we have data from today?
    dfr.loc[end_date]=0 #Add 0s at todays date
    dfr=dfr.resample('D').sum()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [17]:
logger.info('Computing cumulative sums by type...')

for s in df['Category'].unique(): dfr['Volume_'+s]=df[df['Category']==s]['Volume'].resample('D').sum()
dfr=dfr.replace(np.NaN, 0)
for s in df['Category'].unique(): dfr['Volume_Cumulative_'+s]=dfr['Volume_'+s].cumsum()

[05-28 20:33:50] p4237 {<ipython-input-17-a68a068b0f76>:1} INFO - Computing cumulative sums by type...


In [18]:
dfr

,Volume,Volume_Legs,Volume_Back,Volume_Cardio,Volume_Chest,Volume_Arms,Volume_Abs,Volume_Shoulders,Volume_Cumulative_Legs,Volume_Cumulative_Back,Volume_Cumulative_Cardio,Volume_Cumulative_Chest,Volume_Cumulative_Arms,Volume_Cumulative_Abs,Volume_Cumulative_Shoulders
Date,,,,,,,,,,,,,,,
2022-05-09,6380.0,2050.0,100.0,500.0,2380.0,1350.0,0.0,0.0,2050.0,100.0,500.0,2380.0,1350.0,0.0,0.0
2022-05-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2050.0,100.0,500.0,2380.0,1350.0,0.0,0.0
2022-05-11,7525.0,2865.0,1440.0,500.0,100.0,960.0,100.0,1560.0,4915.0,1540.0,1000.0,2480.0,2310.0,100.0,1560.0
2022-05-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4915.0,1540.0,1000.0,2480.0,2310.0,100.0,1560.0
2022-05-13,5870.0,3120.0,280.0,250.0,1920.0,0.0,0.0,300.0,8035.0,1820.0,1250.0,4400.0,2310.0,100.0,1860.0
2022-05-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8035.0,1820.0,1250.0,4400.0,2310.0,100.0,1860.0
2022-05-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8035.0,1820.0,1250.0,4400.0,2310.0,100.0,1860.0
2022-05-16,9850.0,4980.0,100.0,500.0,2620.0,1650.0,0.0,0.0,13015.0,1920.0,1750.0,7020.0,3960.0,100.0,1860.0
2022-05-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13015.0,1920.0,1750.0,7020.0,3960.0,100.0,1860.0


In [19]:
logger.info('Deleting old Analytics data..')
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

[05-28 20:33:56] p4237 {<ipython-input-19-214e3df9818d>:1} INFO - Deleting old Analytics data..


In [20]:
logger.info('Uploading new analytics data...')
l=[]
for s in df['Category'].unique():
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                 'Exercise Volume': dfr.loc[d]['Volume_Cumulative_'+s],
                 'Category': s})
res=a.batch_create(l)

[05-28 20:34:00] p4237 {<ipython-input-20-6841a70e9f65>:1} INFO - Uploading new analytics data...


## General Actions Bro

In [80]:
dst_table_name='[A]Actions'
logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

[05-28 21:17:16] p4237 {<ipython-input-80-5269cd719c88>:2} INFO - Deleting existing [A]Actions data..


In [81]:
logger.info('Connecting to Log, Habits, and Goal table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')
goal_table=Table(api_key, base_id, 'Goals')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-28 21:17:25] p4237 {<ipython-input-81-35f6059d87de>:1} INFO - Connecting to Log, Habits, and Goal table...
[05-28 21:17:25] p4237 {<ipython-input-81-35f6059d87de>:6} INFO - Connecting to [A]Actions table...


In [82]:
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Action' not in o['fields']: continue
    if 'Goal' not in o['fields']: continue
    res.append({'Date': o['fields']['Date'], 
                'Action': o['fields']['Action'],
                'Goal': o['fields']['Goal']})

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()

100%|██████████| 201/201 [00:00<00:00, 227116.14it/s]
[05-28 21:17:26] p4237 {<ipython-input-82-2ceace5eafe0>:10} INFO - Converting to dataframe and resampling...


In [83]:
for g in df['Goal'].unique():
    df['Goal_'+g]=(df['Goal']==g)
dfr=df.resample('D').sum()
dfr=dfr.replace(np.NaN, 0)
for s in df['Goal'].unique(): dfr['Goal_Cumulative_'+s]=dfr['Goal_'+s].cumsum()

In [84]:
dfr.head()

,Goal_Invisible: General,Goal_Productivity,Goal_Personal Development,Goal_Chores,Goal_Ivy,Goal_Finance - $2.5M,Goal_Birthdays,Goal_House,Goal_Fun,Goal_Giving,Goal_Cumulative_Invisible: General,Goal_Cumulative_Productivity,Goal_Cumulative_Personal Development,Goal_Cumulative_Chores,Goal_Cumulative_Ivy,Goal_Cumulative_Finance - $2.5M,Goal_Cumulative_Birthdays,Goal_Cumulative_House,Goal_Cumulative_Fun,Goal_Cumulative_Giving
Date,,,,,,,,,,,,,,,,,,,,
2022-05-09,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
2022-05-10,11,1,1,1,0,0,0,0,0,0,13,1,1,1,0,0,0,0,0,0
2022-05-11,7,0,0,8,1,0,0,0,0,0,20,1,1,9,1,0,0,0,0,0
2022-05-12,2,0,0,0,0,1,2,0,0,0,22,1,1,9,1,1,2,0,0,0
2022-05-13,2,0,0,1,1,0,0,0,0,0,24,1,1,10,2,1,2,0,0,0


In [ ]:
logger.info('Uploading new actions data...')
l=[]
for s in df['Goal'].unique():
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                  'Actions Completed': int(dfr.loc[d]['Goal_'+s]),
                  'Actions Completed Cumulative': int(dfr.loc[d]['Goal_Cumulative_'+s]),
                  'Goal': s})
res=a.batch_create(l)

[05-28 21:18:22] p4237 {<ipython-input-86-7ac096f1e725>:1} INFO - Uploading new actions data...
